In [11]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [12]:
from src.features.extract_features import *
from src.data.dataset import *

In [13]:
data_dir = r"C:\Users\Hagar\Desktop\Biosignal-Analysis-for-VR-Based-Therapy\notebooks\labeled"

feature_set = ['zc', 'ssc', 'wl', 'mav', 'rms']
dataset = EmgDataset(data_dir, win_size=200, win_stride=50, feature_set=feature_set)

In [19]:
emg_features = dataset.extracted_features
labels = dataset.rolled_labels
reps = dataset.rolled_repetition

AttributeError: 'EmgDataset' object has no attribute 'rolled_repetition'

In [ ]:
train_rows = np.isin(reps, ['1','3','4']).ravel()
test_rows = np.isin(reps, ['2']).ravel()

train_emg = emg_features[train_rows]
y_train = labels[train_rows].ravel()

test_emg = emg_features[test_rows]
y_test = labels[test_rows].ravel()

print(f"X_train shape: {train_emg.shape} y_train shape: {y_train.shape}")
print(f"X_test shape: {test_emg.shape} y_test shape: {y_test.shape}")